In [1]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from config import fetch_data


## **CUSTOMER ANALYTICS**

### Customer Overview KPI Cards

In [2]:
kpi_overview_customer_cards_query = "SELECT * FROM gold.analytics_customer_overview"
kpi_overview_customer_cards_df = fetch_data(kpi_overview_customer_cards_query)
df = kpi_overview_customer_cards_df.iloc[0]

fig = make_subplots(
    rows=2, cols=4,
    specs=[[{"type": "indicator"}] * 4] * 2,
    vertical_spacing=0.15,
    horizontal_spacing=0.08
)

kpis = [
    ("Total Customers", df.total_customers, "", "#1f77b4", ","),
    ("Active Customers", df.active_customers, "", "#2ca02c", ","),
    ("Premium Customers", df.premium_customers, "", "#9467bd", ","),
    ("Avg CLV", df.avg_customer_lifetime_value, "$", "#17becf", ",.0f"),

    ("Avg Credit Score", df.avg_credit_score, "", "#ff7f0e", ",.0f"),
    ("High Churn Risk", df.high_churn_risk_customers, "", "#d62728", ","),
    ("Avg Tenure (months)", df.avg_tenure_months, "", "#8c564b", ",.1f"),
    ("Total CLV", df.total_customer_lifetime_value, "$", "#2ca02c", " ,.0f")
]

positions = [(1,1),(1,2),(1,3),(1,4),(2,1),(2,2),(2,3),(2,4)]

for (title, value, suffix, color, fmt), (r,c) in zip(kpis, positions):
    fig.add_trace(
        go.Indicator(
            mode="number",
            value=value,
            title={
                "text": f"<span style='font-size:20px;color:#6b7280'>{title}</span>"
            },
            number={
                "prefix": suffix,
                "valueformat": fmt,
                "font": {"size": 42, "color": color}
            }
        ),
        row=r, col=c
    )

fig.update_layout(
    title={
        "text": "Customer Overview",
        "x": 0.01,
        "font": {"size": 32}
    },
    height=600,
    paper_bgcolor="#0f172a",
    plot_bgcolor="#0f172a",
    title_font_color="white",
    margin=dict(t=100, l=80, r=80, b=80),
    showlegend=False
)

fig.show()

In [3]:
kpi_overview_customer_cards_df

,total_customers,active_customers,inactive_customers,premium_customers,affluent_customers,mass_market_customers,avg_customer_lifetime_value,total_customer_lifetime_value,avg_annual_income,avg_credit_score,high_churn_risk_customers,avg_churn_risk_pct,avg_tenure_months,last_updated
0,21578,16112,5466,5432,5383,5378,50514.92,1.090011e+09,260040.6,550.0,8791,50.14,95.2,2025-12-26 03:41:28.735053+00:00


### Customer Senment Distribution Pie

In [4]:
customer_segment_distribution_pie_query = "SELECT * FROM gold.analytics_customer_by_segment"
customer_segment_distribution_pie_df = fetch_data(customer_segment_distribution_pie_query)

customer_segment_distribution_pie_df['avg_clv'] = customer_segment_distribution_pie_df['avg_clv'].fillna(0)
customer_segment_distribution_pie_df['avg_income'] = customer_segment_distribution_pie_df['avg_income'].fillna(0)

# Build hover string
customer_segment_distribution_pie_df['hover_info'] = (
    "<b>" + customer_segment_distribution_pie_df['customer_segment'] + "</b><br><br>"
    + "Customers: " + customer_segment_distribution_pie_df['customer_count'].map("{:,}".format) + "<br>"
    + "Avg CLV: $" + customer_segment_distribution_pie_df['avg_clv'].map("{:,.2f}".format) + "<br>"
    + "Avg Income: $" + customer_segment_distribution_pie_df['avg_income'].map("{:,.2f}".format)
)

fig = go.Figure(data=[
    go.Pie(
        labels=customer_segment_distribution_pie_df['customer_segment'],
        values=customer_segment_distribution_pie_df['customer_count'],
        hole=0.45,
        textinfo='label+percent',
        textfont_size=13,
        hoverinfo='text',
        hovertext=customer_segment_distribution_pie_df['hover_info'],  
        marker=dict(colors=["#6366f1", "#22c55e", "#06b6d4", "#f59e0b"])
    )
])

fig.update_traces(
    marker=dict(
        colors=["#6366f1", "#22c55e", "#06b6d4", "#f59e0b"]
    )
)

fig.update_layout(
    title="Customer Distribution by Segment",
    height=500,
    paper_bgcolor="#f9fafb",
    plot_bgcolor="#f9fafb",
    showlegend=True,
    legend_title_text="Customer Segment",
    annotations=[dict(text=f"<b>{customer_segment_distribution_pie_df.customer_count.sum():,}</b><br>Customers", x=0.5, y=0.5, font_size=20, showarrow=False)]
)

fig.show()


In [5]:
customer_segment_distribution_pie_df

,customer_segment,customer_count,pct_of_total,avg_clv,avg_income,avg_credit_score,avg_tenure_months,last_updated,hover_info
0,Premium,5432,25.17,50325.09,259673.90,551.0,95.6,2025-12-26 03:41:28.299248+00:00,"<b>Premium</b><br><br>Customers: 5,432<br>Avg ..."
1,Business,5385,24.96,50669.59,259337.86,549.0,96.6,2025-12-26 03:41:28.299248+00:00,"<b>Business</b><br><br>Customers: 5,385<br>Avg..."
2,Affluent,5383,24.95,50635.20,259450.59,549.0,93.7,2025-12-26 03:41:28.299248+00:00,"<b>Affluent</b><br><br>Customers: 5,383<br>Avg..."
3,Mass Market,5378,24.92,50431.39,261705.21,549.0,94.7,2025-12-26 03:41:28.299248+00:00,"<b>Mass Market</b><br><br>Customers: 5,378<br>..."


### Customer Age Distribution Bar

In [6]:
customer_age_distribution_bar_query = "SELECT * FROM gold.analytics_customer_by_age_group ORDER BY age_group"
customer_age_distribution_bar_df = fetch_data(customer_age_distribution_bar_query)

colors = px.colors.qualitative.Set3
fig = go.Figure()

fig.add_trace(go.Bar(
    x=customer_age_distribution_bar_df['age_group'],
    y=customer_age_distribution_bar_df['customer_count'],
    text=customer_age_distribution_bar_df['customer_count'],
    texttemplate='%{text:,}',
    textposition='inside',
    insidetextanchor='middle',
    marker=dict(
        color=colors[:len(customer_age_distribution_bar_df)],
        line=dict(color='rgba(0,0,0,0.15)', width=1)
    ),
    hovertemplate=(
        '<b>Age Group:</b> %{x}<br>'
        '<b>Customers:</b> %{y:,}<br>'
        '<b>Avg CLV:</b> $%{customdata[0]:,.2f}<br>'
        '<b>Avg Income:</b> $%{customdata[1]:,.2f}'
        '<extra></extra>'
    ),
    customdata=customer_age_distribution_bar_df[['avg_clv', 'avg_income']].values
))

fig.update_layout(
    title=dict(
        text="Customer Distribution by Age Group",
        x=0.5,
        font=dict(size=20)
    ),
    xaxis=dict(
        title="Age Group",
        type='category',
        showgrid=False
    ),
    yaxis=dict(
        title="Number of Customers",
        showgrid=True,
        gridcolor='rgba(0,0,0,0.08)'
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=520,
    bargap=0.35
)

fig.show()

In [7]:
customer_age_distribution_bar_df

,age_group,customer_count,pct_of_total,avg_clv,avg_income,active_count,last_updated
0,18-24,2864,13.27,50874.27,263976.50,2105,2025-12-26 03:41:27.705293+00:00
1,25-34,4144,19.20,50712.21,260745.96,3158,2025-12-26 03:41:27.705293+00:00
2,35-44,4185,19.39,50667.25,257505.14,3149,2025-12-26 03:41:27.705293+00:00
3,45-54,4157,19.26,49908.82,258554.78,3055,2025-12-26 03:41:27.705293+00:00
4,55-64,4155,19.26,50455.85,260094.32,3136,2025-12-26 03:41:27.705293+00:00
5,65+,2073,9.61,50650.38,261183.33,1509,2025-12-26 03:41:27.705293+00:00


### Customer Geography Map

In [8]:
customer_geography_map_query = "SELECT * FROM gold.analytics_customer_by_geography ORDER BY customer_count DESC LIMIT 25"
customer_geography_map_df = fetch_data(customer_geography_map_query)

fig = px.scatter_geo(
    customer_geography_map_df,
    locations='state',
    locationmode='USA-states',
    size='customer_count',
    color='avg_clv',
    hover_name='state',
    hover_data={
        'customer_count': ':,',
        'avg_clv': ':$,.2f',
        'avg_income': ':$,.2f',
        'state': False
    },
    scope='usa',
    title='Customer Distribution by State (Top 25)',
    color_continuous_scale='Viridis',
    labels={'avg_clv': 'Avg CLV', 'customer_count': 'Customers'}
)

fig.update_layout(height=600)
fig.show()

In [9]:
customer_geography_map_df

,state,city,customer_count,pct_of_total,avg_clv,avg_income,active_count,last_updated
0,TX,HOUSTON,63,0.29,46822.35,280437.73,45,2025-12-26 03:41:28.012323+00:00
1,NY,NEW YORK,47,0.22,54274.85,260094.89,36,2025-12-26 03:41:28.012323+00:00
2,CA,LOS ANGELES,40,0.19,53162.83,282021.75,31,2025-12-26 03:41:28.012323+00:00
3,TX,SAN ANTONIO,38,0.18,47711.53,272815.68,35,2025-12-26 03:41:28.012323+00:00
4,MN,MINNEAPOLIS,37,0.17,47694.95,241249.76,24,2025-12-26 03:41:28.012323+00:00
5,PA,PHILADELPHIA,36,0.17,44858.67,243579.39,26,2025-12-26 03:41:28.012323+00:00
6,DC,WASHINGTON,36,0.17,61292.78,248774.89,29,2025-12-26 03:41:28.012323+00:00
7,TX,DALLAS,35,0.16,45938.37,262000.97,28,2025-12-26 03:41:28.012323+00:00
8,CO,DENVER,34,0.16,42044.09,253108.59,22,2025-12-26 03:41:28.012323+00:00
9,PA,PITTSBURGH,33,0.15,53597.36,263273.97,20,2025-12-26 03:41:28.012323+00:00


## **TRANSACTION ANALYTICS**

### Transaction Summary KPI

In [10]:
transaction_summary_kpi_query = "SELECT * FROM gold.analytics_transaction_summary"
transaction_summary_kpi_df = fetch_data(transaction_summary_kpi_query)

fig = make_subplots(
    rows=2, cols=4,
    specs=[[{"type": "indicator"}] * 4] * 2,
    subplot_titles=(
        "Total Transactions", "Total Volume", "Avg Amount", "Fraud Rate",
        "Online Trans", "Mobile Trans", "Branch Trans", "ATM Trans"
    )
)

metrics = [
    (transaction_summary_kpi_df['total_transactions'].iloc[0], "", "#636EFA"),
    (transaction_summary_kpi_df['total_transaction_volume'].iloc[0], "$", "#00CC96"),
    (transaction_summary_kpi_df['avg_transaction_amount'].iloc[0], "$", "#AB63FA"),
    (transaction_summary_kpi_df['fraud_rate_pct'].iloc[0], "%", "#EF553B"),
    (transaction_summary_kpi_df['online_transactions'].iloc[0], "", "#19D3F3"),
    (transaction_summary_kpi_df['mobile_transactions'].iloc[0], "", "#FFA15A"),
    (transaction_summary_kpi_df['branch_transactions'].iloc[0], "", "#B6E880"),
    (transaction_summary_kpi_df['atm_transactions'].iloc[0], "", "#FF6692")
]

positions = [
    (1, 1), (1, 2), (1, 3), (1, 4),
    (2, 1), (2, 2), (2, 3), (2, 4)
]

for (value, suffix, color), (row, col) in zip(metrics, positions):
    fig.add_trace(
        go.Indicator(
            mode="number",
            value=value,
            number={
                "prefix": "$" if suffix == "$" else "",
                "suffix": "%" if suffix == "%" else "",
                "valueformat": ",.2f" if suffix in ["$", "%"] else ",.0f",
                "font": {"size": 42, "color": color}
            }
        ),
        row=row, col=col
    )

fig.update_layout(
    title=dict(
        text="Transaction Overview Dashboard",
        x=0.5,
        font=dict(size=32),
    ),
    height=550,
    showlegend=False,
    paper_bgcolor="white",
    plot_bgcolor="white",
)

fig.show()

In [11]:
transaction_summary_kpi_df

,total_transactions,total_transaction_volume,avg_transaction_amount,debit_count,credit_count,total_debits,total_credits,online_transactions,mobile_transactions,branch_transactions,atm_transactions,fraud_transactions,total_fraud_amount,fraud_rate_pct,high_value_transactions,international_transactions,last_updated
0,126042,1.988211e+08,1577.42,72088,53954,81206265.2,1.176149e+08,24999,25262,25247,25342,117,181113.41,0.09,0,62940,2025-12-26 03:41:36.103328+00:00


### Transaction Channel Distribution

In [12]:
transaction_channel_distribution_query = "SELECT * FROM gold.analytics_transaction_by_channel"
transaction_channel_distribution_df = fetch_data(transaction_channel_distribution_query)

transaction_channel_distribution_df['hover_info'] = (
    'Share: ' + transaction_channel_distribution_df['pct_of_total'].astype(str) + '<br>' +
    'Total Volume: $' + transaction_channel_distribution_df['total_volume'].map('{:,.2f}'.format) + '<br>' +
    'Avg Amount: $' + transaction_channel_distribution_df['avg_amount'].map('{:,.2f}'.format)
)

colors = px.colors.qualitative.Pastel

fig = go.Figure()

fig.add_trace(
    go.Pie(
        labels=transaction_channel_distribution_df['channel'],
        values=transaction_channel_distribution_df['transaction_count'],
        hole=0.6,
        textinfo='label+percent',
        textfont=dict(size=13),
        marker=dict(
            colors=colors,
            line=dict(color='white', width=2)
        ),
        pull=[0.03] * len(transaction_channel_distribution_df),
        hovertext=customer_segment_distribution_pie_df['hover_info'],
    )
)

fig.update_layout(
    title=dict(
        text="Transaction Distribution by Channel",
        x=0.5,
        font=dict(size=20)
    ),
    height=520,
    paper_bgcolor='white',
    plot_bgcolor='white',
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.15,
        xanchor="center",
        x=0.5
    ),
    annotations=[
        dict(
            text="<b>Transactions</b>",
            x=0.5,
            y=0.5,
            font_size=16,
            showarrow=False
        )
    ]
)

fig.show()

In [13]:
transaction_channel_distribution_df

,channel,transaction_count,pct_of_total,total_volume,avg_amount,unique_customers,fraud_count,last_updated,hover_info
0,ATM,25342,20.11,40057508.11,1580.68,12772,21,2025-12-26 03:41:35.859233+00:00,"Share: 20.11<br>Total Volume: $40,057,508.11<b..."
1,Mobile,25262,20.04,39920254.26,1580.25,12875,26,2025-12-26 03:41:35.859233+00:00,"Share: 20.04<br>Total Volume: $39,920,254.26<b..."
2,Branch,25247,20.03,39720159.29,1573.26,12866,26,2025-12-26 03:41:35.859233+00:00,"Share: 20.03<br>Total Volume: $39,720,159.29<b..."
3,POS,25192,19.99,39745861.27,1577.72,12881,22,2025-12-26 03:41:35.859233+00:00,"Share: 19.99<br>Total Volume: $39,745,861.27<b..."
4,Online,24999,19.83,39377333.62,1575.16,12828,22,2025-12-26 03:41:35.859233+00:00,"Share: 19.83<br>Total Volume: $39,377,333.62<b..."


### Transaction Category Treemap 

In [14]:
transaction_category_treemap_query = "SELECT * FROM gold.analytics_transaction_by_category ORDER BY total_volume DESC LIMIT 25"
transaction_category_treemap_df = fetch_data(transaction_category_treemap_query)


fig = px.treemap(
    transaction_category_treemap_df,
    path=['merchant_category'],
    values='total_volume',
    color='avg_amount',
    hover_data={
        'transaction_count': ':,',
        'total_volume': ':$,.2f',
        'avg_amount': ':$,.2f',
        'unique_customers': ':,'
    },
    title='Transaction Volume by Merchant Category (Top 25)',
    color_continuous_scale='RdYlGn',
    labels={'total_volume': 'Total Volume', 'avg_amount': 'Avg Amount'}
)

fig.update_layout(height=600)

In [15]:
transaction_category_treemap_df

,merchant_category,transaction_count,pct_of_total,total_volume,avg_amount,unique_customers,last_updated
0,Retail,12871,10.21,20433773.03,1587.58,8751,2025-12-26 03:41:35.625606+00:00
1,Healthcare,12744,10.11,20405777.79,1601.21,8672,2025-12-26 03:41:35.625606+00:00
2,Entertainment,12650,10.04,20147677.28,1592.70,8659,2025-12-26 03:41:35.625606+00:00
3,Restaurant,12657,10.04,19970524.00,1577.82,8643,2025-12-26 03:41:35.625606+00:00
4,Travel,12661,10.05,19882347.66,1570.36,8705,2025-12-26 03:41:35.625606+00:00
5,Grocery,12646,10.03,19778051.32,1563.98,8719,2025-12-26 03:41:35.625606+00:00
6,Services,12582,9.98,19697595.60,1565.54,8634,2025-12-26 03:41:35.625606+00:00
7,Online Shopping,12431,9.86,19546449.76,1572.40,8613,2025-12-26 03:41:35.625606+00:00
8,Utilities,12424,9.86,19479837.11,1567.92,8714,2025-12-26 03:41:35.625606+00:00
9,Gas Station,12376,9.82,19479083.00,1573.94,8556,2025-12-26 03:41:35.625606+00:00


### Daily Transaction Trend

In [16]:
daily_transaction_trend_query = "SELECT * FROM gold.analytics_daily_transaction_trend ORDER BY transaction_date"
daily_transaction_trend_df = fetch_data(daily_transaction_trend_query)


daily_transaction_trend_df['transaction_date'] = pd.to_datetime(daily_transaction_trend_df['transaction_date'])

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Transaction Count", "Transaction Volume"),
    vertical_spacing=0.12
)

# Transaction count
fig.add_trace(
    go.Scatter(
        x=daily_transaction_trend_df['transaction_date'],
        y=daily_transaction_trend_df['transaction_count'],
        mode='lines',
        name='Count',
        line=dict(color='#636EFA', width=2),
        fill='tozeroy',
        hovertemplate='Date: %{x}<br>Count: %{y:,}<extra></extra>'
    ),
    row=1, col=1
)

# Transaction volume
fig.add_trace(
    go.Scatter(
        x=daily_transaction_trend_df['transaction_date'],
        y=daily_transaction_trend_df['total_volume'],
        mode='lines',
        name='Volume',
        line=dict(color='#00CC96', width=2),
        fill='tozeroy',
        hovertemplate='Date: %{x}<br>Volume: $%{y:,.2f}<extra></extra>'
    ),
    row=2, col=1
)

# Add weekend shading
weekends = daily_transaction_trend_df[daily_transaction_trend_df['is_weekend'] == True]
for idx, row in weekends.iterrows():
    for subplot_row in [1, 2]:
        fig.add_vrect(
            x0=row['transaction_date'], x1=row['transaction_date'] +
            pd.Timedelta(days=1),
            fillcolor="lightgray", opacity=0.2, layer="below", line_width=0,
            row=subplot_row, col=1
        )

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Transaction Count", row=1, col=1)
fig.update_yaxes(title_text="Transaction Volume ($)", row=2, col=1)

fig.update_layout(
    title="Daily Transaction Trends (Last 365 Days)",
    height=700,
    showlegend=False
)

fig.show()

In [17]:
daily_transaction_trend_df

,transaction_date,day_name,is_weekend,transaction_count,total_volume,avg_amount,active_customers,fraud_count,last_updated


### Monthly Transaction Trend

In [18]:
monthly_transaction_trend_query = "SELECT * FROM gold.analytics_monthly_transaction_trend ORDER BY year, month"
monthly_transaction_trend_df = fetch_data(monthly_transaction_trend_query)

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Bar chart for transaction count
fig.add_trace(
    go.Bar(
        x=monthly_transaction_trend_df['year_month'],
        y=monthly_transaction_trend_df['transaction_count'],
        name='Transaction Count',
        marker_color='#636EFA',
        hovertemplate='%{x}<br>Count: %{y:,}<extra></extra>'
    ),
    secondary_y=False
)

# Line chart for fraud rate
fig.add_trace(
    go.Scatter(
        x=monthly_transaction_trend_df['year_month'],
        y=monthly_transaction_trend_df['fraud_rate_pct'],
        name='Fraud Rate %',
        mode='lines+markers',
        line=dict(color='red', width=3),
        yaxis='y2',
        hovertemplate='%{x}<br>Fraud Rate: %{y:.2f}%<extra></extra>'
    ),
    secondary_y=True
)

fig.update_xaxes(title_text="Month")
fig.update_yaxes(title_text="Transaction Count", secondary_y=False)
fig.update_yaxes(title_text="Fraud Rate (%)", secondary_y=True)

fig.update_layout(
    title="Monthly Transaction Trends (Last 24 Months)",
    height=500,
    hovermode='x unified'
)

fig.show()

In [19]:
monthly_transaction_trend_df

,year,month,month_name,year_month,transaction_count,total_volume,avg_amount,active_customers,fraud_count,fraud_rate_pct,last_updated


## **ACCOUNT ANALYTICS**

### Account Summary Gague 

In [20]:
account_summary_gauge_query = "SELECT * FROM gold.analytics_account_summary"
account_summary_gauge_df = fetch_data(account_summary_gauge_query)

avg_util = account_summary_gauge_df['avg_credit_utilization_pct'].iloc[0]
past_due = account_summary_gauge_df['past_due_rate_pct'].iloc[0]
active_pct = (account_summary_gauge_df['active_accounts'].iloc[0] / account_summary_gauge_df['total_accounts'].iloc[0]) * 100
avg_age = account_summary_gauge_df['avg_account_age_months'].iloc[0]

# =========================
# Create Subplots
# =========================
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "indicator"}, {"type": "indicator"}],
           [{"type": "indicator"}, {"type": "indicator"}]],
    subplot_titles=[
        "💳 Credit Utilization",
        "⚠️ Past Due Rate",
        "✅ Active Accounts",
        "📆 Avg Account Age"
    ]
)

# =========================
# Credit Utilization
# =========================
fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=avg_util,
        number={"suffix": "%", "font": {"size": 36}},
        gauge={
            "axis": {"range": [0, 100]},
            "bar": {"color": "#1f77b4"},
            "bgcolor": "#F5F5F5",
            "steps": [
                {"range": [0, 30], "color": "#B7E4C7"},
                {"range": [30, 70], "color": "#FFE066"},
                {"range": [70, 100], "color": "#FF6B6B"}
            ],
            "threshold": {
                "line": {"color": "#C92A2A", "width": 4},
                "value": 80
            }
        }
    ),
    row=1, col=1
)

# =========================
# Past Due Rate
# =========================
fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=past_due,
        number={"suffix": "%", "font": {"size": 36}},
        gauge={
            "axis": {"range": [0, 10]},
            "bar": {"color": "#C92A2A"},
            "bgcolor": "#F5F5F5",
            "steps": [
                {"range": [0, 2], "color": "#B7E4C7"},
                {"range": [2, 5], "color": "#FFE066"},
                {"range": [5, 10], "color": "#FF6B6B"}
            ]
        }
    ),
    row=1, col=2
)

# =========================
# Active Accounts %
# =========================
fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=active_pct,
        number={"suffix": "%", "font": {"size": 36}},
        gauge={
            "axis": {"range": [0, 100]},
            "bar": {"color": "#2F9E44"},
            "bgcolor": "#F5F5F5",
            "steps": [
                {"range": [0, 70], "color": "#FF6B6B"},
                {"range": [70, 85], "color": "#FFE066"},
                {"range": [85, 100], "color": "#B7E4C7"}
            ]
        }
    ),
    row=2, col=1
)

# =========================
# Avg Account Age
# =========================
fig.add_trace(
    go.Indicator(
        mode="number",
        value=avg_age,
        number={
            "suffix": " months",
            "font": {"size": 40}
        }
    ),
    row=2, col=2
)

# =========================
# Layout Styling
# =========================
fig.update_layout(
    title={
        "text": "📊 Account Health Overview",
        "x": 0.5,
        "xanchor": "center",
        "font": {"size": 24}
    },
    height=720,
    template="plotly_white",
    font={"family": "Inter, Arial"},
    margin=dict(t=90, b=40, l=40, r=40)
)

fig.show()

In [21]:
account_summary_gauge_df

,total_accounts,active_accounts,closed_accounts,dormant_accounts,total_active_balance,avg_account_balance,total_available_balance,total_credit_limit,avg_credit_utilization_pct,past_due_accounts,near_limit_accounts,past_due_rate_pct,avg_account_age_months,last_updated
0,64811,38779,18261,12926,-1.286923e+09,-33186.08,-1.465617e+09,1.951254e+08,71.63,0,2397,0.0,81.8,2025-12-26 03:41:26.596355+00:00


### Account Product Distribution

In [22]:
account_product_distribution_query = "SELECT * FROM gold.analytics_account_by_product ORDER BY account_count DESC"
account_product_distribution_df = fetch_data(account_product_distribution_query)

account_product_distribution_df["inactive_accounts"] = account_product_distribution_df["account_count"] - account_product_distribution_df["active_accounts"]
account_product_distribution_df["active_pct"] = (account_product_distribution_df["active_accounts"] / account_product_distribution_df["account_count"]) * 100

# =========================
# Build Figure
# =========================
fig = go.Figure()

# Active Accounts
fig.add_trace(go.Bar(
    name="Active Accounts",
    y=account_product_distribution_df["product_name"],
    x=account_product_distribution_df["active_accounts"],
    orientation="h",
    marker_color="#2F9E44",
    hovertemplate=(
        "<b>%{y}</b><br>"
        "Active: %{x:,}<br>"
        "Active %: %{customdata:.1f}%<extra></extra>"
    ),
    customdata=account_product_distribution_df["active_pct"]
))

# Inactive Accounts
fig.add_trace(go.Bar(
    name="Inactive Accounts",
    y=account_product_distribution_df["product_name"],
    x=account_product_distribution_df["inactive_accounts"],
    orientation="h",
    marker_color="#FF6B6B",
    hovertemplate=(
        "<b>%{y}</b><br>"
        "Inactive: %{x:,}<extra></extra>"
    )
))

# =========================
# Layout Styling
# =========================
fig.update_layout(
    title={
        "text": "📦 Account Distribution by Product",
        "x": 0.5,
        "xanchor": "center",
        "font": {"size": 22}
    },
    barmode="stack",
    template="plotly_white",
    height=520,
    font={"family": "Inter, Arial"},
    xaxis={
        "title": "Number of Accounts",
        "showgrid": True,
        "zeroline": False
    },
    yaxis={
        "title": "",
        "categoryorder": "total ascending"
    },
    legend={
        "orientation": "h",
        "y": -0.15,
        "x": 0.5,
        "xanchor": "center"
    },
    margin=dict(t=80, l=120, r=40, b=80),
    hovermode="y unified"
)

fig.show()

In [23]:
account_product_distribution_df

,product_name,category,product_line,account_count,pct_of_total,active_accounts,total_balance,avg_balance,unique_customers,last_updated,inactive_accounts,active_pct
0,Investment Account,INVESTMENT,Wealth Management,6608,10.20,4018,1.014766e+09,252555.03,5793,2025-12-26 03:41:26.594330+00:00,2590,60.805085
1,Personal Loan,LOAN,Lending,6518,10.06,3924,-9.830889e+08,-250532.34,5727,2025-12-26 03:41:26.594330+00:00,2594,60.202516
2,Auto Loan,LOAN,Lending,6514,10.05,3916,-1.002040e+09,-255883.47,5674,2025-12-26 03:41:26.594330+00:00,2598,60.116672
3,Business Checking,DEPOSIT,Banking,6511,10.05,3874,1.908374e+08,49261.08,5679,2025-12-26 03:41:26.594330+00:00,2637,59.499309
4,Mortgage,LOAN,Lending,6508,10.04,3923,-1.014395e+09,-258576.22,5671,2025-12-26 03:41:26.594330+00:00,2585,60.279656
5,Checking Account,DEPOSIT,Banking,6496,10.02,3889,1.936200e+08,49786.57,5685,2025-12-26 03:41:26.594330+00:00,2607,59.867611
6,Premium Credit Card,CREDIT,Lending,6466,9.98,3859,-3.414289e+07,-8847.60,5659,2025-12-26 03:41:26.594330+00:00,2607,59.681410
7,Money Market Account,DEPOSIT,Banking,6452,9.96,3839,1.931356e+08,50308.83,5633,2025-12-26 03:41:26.594330+00:00,2613,59.500930
8,Credit Card,CREDIT,Lending,6384,9.85,3806,-3.362391e+07,-8834.45,5601,2025-12-26 03:41:26.594330+00:00,2578,59.617794
9,Savings Account,DEPOSIT,Banking,6354,9.80,3731,1.880077e+08,50390.70,5578,2025-12-26 03:41:26.594330+00:00,2623,58.718917


### Account Balanace Histogram 

In [24]:
account_balance_histogram_query = "SELECT * FROM gold.analytics_account_balance_distribution"
account_balance_histogram_df = fetch_data(account_balance_histogram_query)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=account_balance_histogram_df["balance_category"],
    y=account_balance_histogram_df["account_count"],
    marker_color="#845EF7",
    text=account_balance_histogram_df["pct_of_total"].apply(lambda x: f"{x:.1f}%"),
    textposition="outside",
    hovertemplate=(
        "<b>%{x}</b><br>"
        "Accounts: %{y:,}<br>"
        "Avg Balance: $%{customdata[0]:,.2f}<br>"
        "Total Balance: $%{customdata[1]:,.2f}"
        "<extra></extra>"
    ),
    customdata=account_balance_histogram_df[["avg_balance", "total_balance"]].values
))

# =========================
# Layout Styling
# =========================
fig.update_layout(
    title={
        "text": "💰 Account Balance Distribution",
        "x": 0.5,
        "xanchor": "center",
        "font": {"size": 22}
    },
    template="plotly_white",
    height=520,
    font={"family": "Inter, Arial"},
    xaxis={
        "title": "Balance Range",
        "tickangle": -20,
        "showgrid": False
    },
    yaxis={
        "title": "Number of Accounts",
        "showgrid": True,
        "zeroline": False
    },
    margin=dict(t=80, l=60, r=40, b=90)
)

fig.show()

In [25]:
account_balance_histogram_df

,balance_category,account_count,pct_of_total,total_balance,avg_balance,min_balance,max_balance,last_updated
0,Negative,19428,50.10,-3.067290e+09,-157879.86,-499951.15,-0.88,2025-12-26 03:41:26.585948+00:00
1,Low,155,0.40,8.264187e+04,533.17,102.32,996.33,2025-12-26 03:41:26.585948+00:00
2,Medium,1420,3.66,7.863324e+06,5537.55,1000.57,9997.26,2025-12-26 03:41:26.585948+00:00
3,High,14534,37.48,7.965210e+08,54803.97,10001.74,99998.07,2025-12-26 03:41:26.585948+00:00
4,Very High,3242,8.36,9.758999e+08,301017.86,100690.86,499940.49,2025-12-26 03:41:26.585948+00:00


## **FRAUD & LOAN ANALYTICS**

### Fraud Summary Funnel

In [26]:
fraud_summary_funnel_query = "SELECT * FROM gold.analytics_fraud_summary"
fraud_summary_funnel_df = fetch_data(fraud_summary_funnel_query)

stages = [
    "🚨 Total Alerts",
    "🔍 Resolved Alerts",
    "❌ Confirmed Fraud",
    "✅ False Positives"
]

values = [
    fraud_summary_funnel_df["total_alerts"].iloc[0],
    fraud_summary_funnel_df["resolved_alerts"].iloc[0],
    fraud_summary_funnel_df["confirmed_fraud"].iloc[0],
    fraud_summary_funnel_df["false_positives"].iloc[0]
]

# =========================
# Build Funnel
# =========================
fig = go.Figure(go.Funnel(
    y=stages,
    x=values,
    textinfo="value+percent initial",
    textfont={"size": 14},
    marker={
        "color": [
            "#4C6EF5",  # alerts (blue)
            "#15AABF",  # resolved (teal)
            "#FA5252",  # fraud (red)
            "#51CF66"   # false positive (green)
        ]
    },
    hovertemplate=(
        "<b>%{y}</b><br>"
        "Cases: %{x:,}<br>"
        "Share of Alerts: %{percentInitial:.1%}"
        "<extra></extra>"
    )
))

# =========================
# Layout Styling
# =========================
fig.update_layout(
    title={
        "text": "🛡️ Fraud Alert Resolution Funnel",
        "x": 0.5,
        "xanchor": "center",
        "font": {"size": 22}
    },
    template="plotly_white",
    height=520,
    font={"family": "Inter, Arial"},
    margin=dict(t=80, l=80, r=80, b=40)
)

fig.show()

In [27]:
fraud_summary_funnel_df

,total_alerts,resolved_alerts,confirmed_fraud,false_positives,total_recovered,avg_recovered,avg_resolution_days,fraud_confirmation_rate_pct,false_positive_rate_pct,last_updated
0,117,69,22,22,-636.67,-5.44,15.3,31.88,18.8,2025-12-26 03:41:31.562022+00:00


### Load Payment Waterfall

In [28]:
loan_payment_waterfall_query = "SELECT * FROM gold.analytics_loan_payment_summary"
loan_payment_waterfall_df = fetch_data(loan_payment_waterfall_query)

total = loan_payment_waterfall_df["total_payments"].iloc[0]
late = loan_payment_waterfall_df["late_payments"].iloc[0]
missed = loan_payment_waterfall_df["missed_payments"].iloc[0]
on_time = loan_payment_waterfall_df["full_payments"].iloc[0]

# =========================
# Build Waterfall
# =========================
fig = go.Figure(go.Waterfall(
    name="Payment Flow",
    orientation="v",
    measure=["relative", "relative", "relative", "total"],
    x=[
        "Total Payments",
        "Late Payments",
        "Missed Payments",
        "On-Time Payments"
    ],
    y=[
        total,
        -late,
        -missed,
        0  # auto-calculated total
    ],
    text=[
        f"{total:,}",
        f"-{late:,}",
        f"-{missed:,}",
        f"{on_time:,}"
    ],
    textposition="outside",
    increasing={"marker": {"color": "#2F9E44"}},
    decreasing={"marker": {"color": "#FA5252"}},
    totals={"marker": {"color": "#4C6EF5"}},
    connector={"line": {"color": "#ADB5BD"}}
))

# =========================
# Layout Styling
# =========================
fig.update_layout(
    title={
        "text": "💳 Loan Payment Performance Breakdown",
        "x": 0.5,
        "xanchor": "center",
        "font": {"size": 22}
    },
    template="plotly_white",
    height=520,
    font={"family": "Inter, Arial"},
    yaxis={
        "title": "Number of Payments",
        "showgrid": True,
        "zeroline": False
    },
    margin=dict(t=80, l=60, r=40, b=60),
    hovermode="x unified"
)

fig.show()

In [29]:
loan_payment_waterfall_df

,total_payments,full_payments,late_payments,missed_payments,total_scheduled,total_actual,total_late_fees,total_outstanding,late_payment_rate_pct,missed_payment_rate_pct,avg_days_late,last_updated
0,559287,265731,83644,28006,2.835571e+09,2.692611e+09,3135710.45,1.063099e+11,14.96,5.01,1.2,2025-12-26 03:41:31.793648+00:00
